In [1]:
###############################################  set environment  ###############################################
# pd.set_option('display.height', 1000)
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

In [13]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math as m
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import time

###############################################  read data  ###############################################
def read_csv():
    #train = pd.read_csv('dss-regression-datasets/project-sberbank-housing-market-data/train.csv')
#     train = pd.read_csv('treated_train_df.csv')
#     train = pd.read_csv('train_df_treated_20171021.csv')
    train = pd.read_csv('train_tr_byear_sta_20171023.csv')

#     m_train = pd.read_csv('dss-regression-datasets/project-sberbank-housing-market-data/macro.csv')   
    m_train = pd.read_csv('macro_train_imputated_df_20171023.csv')
    return train, m_train

###############################################  encoding part  ###############################################
#Encoding Categorical Variables
def encode_cat(df):
    #get categorical(type='object') variable
    cat_var_df = df.select_dtypes(include=['object']).copy()
    
    
    cleanup_nums = {"product_type":{'OwnerOccupier' : 0, 'Investment' : 1},                
               "ecology": {'no data' : 0, 'poor' : 1, 'satisfactory' : 2, 'good' : 3, 'excellent' : 4}}
    
    #dummy encoding
    for i in cat_var_df.columns:        
        if cat_var_df[i].unique()[0] in ['no', 'yes']:
            cleanup_nums[i] = {'no':0, 'yes':1}
    
    #sub_area encoding
    sub_index = 0
    sub_area_enc_dict = {}
    for key in set(cat_var_df['sub_area'].values):
        sub_area_enc_dict[key] = sub_index
        sub_index += 1
    cleanup_nums['sub_area'] = sub_area_enc_dict
    
    try:
        cat_var_df.replace(cleanup_nums, inplace=True)
    except:
        pass
    
    df = df.drop(cat_var_df.columns, axis=1)
    df = pd.concat([df, cat_var_df], axis=1)
    return df

#encode macro cat variable
def m_encode_cat(df):
    #get categorical(type='object') variable
    cat_var_df = df.select_dtypes(include=['object']).copy()
    
    
    cleanup_nums = {"child_on_acc_pre_school":{'#!' : 0, '3,013' : 1, '7,311' : 2, '16,765' : 3, '45,713' : 4},                
                "modern_education_share": {'90,92' : 1, '93,08' : 1, '95,4918' : 2},
                "old_education_build_share": {'23,14' : 1, '25,47' : 1, '8,2517' : 2}}
           
    try:
        cat_var_df.replace(cleanup_nums, inplace=True)
    except:
        pass
    
    df = df.drop(cat_var_df.columns, axis=1)
    df = pd.concat([df, cat_var_df], axis=1)
    return df

#encoding numeric var
def encode_num(df):
    df['build_year']  = df['build_year'] - 1900
    return df

###############################################  outliers handling part  ###############################################
#dropping outliers by fox_criteria
def dropping_outliers1(x, y):
    x1 = sm.add_constant(x)
    model = sm.OLS(y, x1)
    result = model.fit()
    influence = result.get_influence()
    cooks_d2, pvals = influence.cooks_distance
    fox_cr = 4 / (len(y) - len(x.columns) -1)
    idx = np.where(cooks_d2 > fox_cr)[0]
    x = x.drop(x.index[idx])
    y = y.drop(y.index[idx])
    print("Function dropping_outliers :" + str(len(idx)) + ' rows have beed deleted')
    return x, y

###############################################  feature selection func part  ###############################################
#backward greedy elimination
def backw_eli(x, y, lim_ = 0.005):
    result_df_reg1 = pd.DataFrame()
    num_high_pvalue = 1000    
    x1 = x
    while num_high_pvalue > 0:
        x1 = sm.add_constant(x1)
        model = sm.OLS(y, x1).fit()
        
        sorted_pvalues = model.pvalues.sort_values(ascending=False)
        if sorted_pvalues.index[0] == 'const':
            x1 = x1.drop(sorted_pvalues.index[1], 1)
        else:
            x1 = x1.drop(sorted_pvalues.index[0], 1)
            
        num_high_pvalue = len(model.pvalues[model.pvalues >= lim_])
        
    model = sm.OLS(y, x1)
    result = model.fit()
        
    return x1, y, result, model

# input : x(numeric var), y(dep var), output: x(rank vectors), y(dep_var)
def serial_eli_lim_pval(x, y, ser_1 = 0.0001, ser_2 = 0.6, ser_3 = 0.0005):
    result_x = []
    
    x1 = x
    remain_var_num = 10    
    while remain_var_num > 0:
        x1 = sm.add_constant(x1)
        result = sm.OLS(y, x1, hasconst=1).fit()
        
        time.sleep(5)
        #get lowest p-val, delete if in case of const
        sorted_pvalues = result.pvalues.sort_values(ascending=True)
#         if 'const' in sorted_pvalues.index:
        sorted_pvalues = sorted_pvalues.drop(['const'], axis=0)
        sorted_pvalues_lst = list(sorted_pvalues.index)
        pval_num1_name = sorted_pvalues_lst[0]
        pval_num1_value = sorted_pvalues[0]
        
        #check pval, if it is high enough just delete it and back to while loop
        if pval_num1_value > ser_1:
            x1 = x1.drop([pval_num1_name, 'const'], axis=1)
            remain_var_num = len(x1.columns)            
#             if 'const' in x1.columns:
#                 x1 = x1.drop([pval_num1_name, 'const'], axis=1)
#                 remain_var_num = len(x1.columns)
#             else:
#                 x1 = x1.drop([pval_num1_name], axis=1)
#                 remain_var_num = len(x1.columns)
            
        #
        else:  
#             if 'const' in x1.columns:
            x1 = x1.drop(['const'], axis=1)
            x1_cor = x1.corr()
            x1_cor_eli_lst = list(x1_cor[abs(x1_cor[pval_num1_name]) > ser_2].index)

            if pval_num1_name in x1_cor_eli_lst:
                x1_cor_eli_lst.remove(pval_num1_name)
            x1_pval_eli_lst = list(sorted_pvalues[x1_cor_eli_lst][sorted_pvalues[x1_cor_eli_lst] > ser_3].index)
            x1 = x1.drop([*x1_pval_eli_lst, pval_num1_name], axis=1)

            result_x.append(pval_num1_name)
            remain_var_num = len(x1.columns)
    
    x = x[result_x]
    x1 = sm.add_constant(x)
    model = sm.OLS(y, x1)
    result = model.fit()
    
    return x, y, result, model

#automation of r-style from_formula
def r_style_from_formula_enc_included(num_x, cat_x, y, const=1, scal=0): 
    y_df = pd.DataFrame(y)
    if scal == 0:
        scale_st = ""; scale_en = ""
    else:
        scale_st = "scale("; scale_en = ")"
    
    if const == 0:
        constadd = " + 0"  
    else:
        constadd = ""        
       
    model_str = "price_doc ~ "         
    for i, column in enumerate(num_x.columns):
        if i == 0:
            prefix = ""
        else:
            prefix = " + "
        model_str += prefix + scale_st + column + scale_en
    for i, column in enumerate(cat_x.columns):
        prefix = " + "
        model_str += prefix + "C(" + column + ")"

    df = pd.concat([num_x, cat_x, y_df], axis=1) 
    model_str += constadd
    model = sm.OLS.from_formula(model_str, data=df)
    result = model.fit()
    
    return result, model

#print(poly_string('aaa', 12, True))
#print(poly_string('aaa', 12, False))
def poly_string(var_, deg, scale_=True):
    
    poly_sum = 'scale({})'.format(var_) if scale_ else var_
    for pwr in range(2, deg+1):
        nth_deg = ' + scale(I({}**{}))'.format(var_, pwr) if scale_\
                  else ' + I({}**{})'.format(var_, pwr)
        poly_sum += nth_deg
    return poly_sum

###############################################  result calculation part  ###############################################
#rmsle calculation
def rmsle(y, h): 
    """
    Compute the Root Mean Squared Log Error for hypthesis h and targets y
     
    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())

In [9]:
###############################################  controler part  ###############################################
def auto_eli_and_result_print(x, y, model = 0, ser_1 = 0.0001, ser_2 = 0.6, ser_3 = 0.0005, lim_=0.05):
    if model == 0:
        x_treat, y, result, model = serial_eli_lim_pval(x, y, ser_1, ser_2, ser_3)
    else:
        x_treat, y, result, model = backw_eli(x, y, lim_)
#     x_treat_1 = sm.add_constant(x_treat)
#     y_predict = result.predict(x_treat_1)
#     print('RMSE: ', np.sqrt(metrics.mean_squared_error(y, y_predict)))
#     print('RMSLE', rmsle(y, y_predict))
#     print('R2 : ', result.rsquared, '\n')
#     print('Adj. R2 : ', result.rsquared_adj, '\n')
#     print(result.summary())
    return x_treat, y, result, model

###############################################  operation part  ###############################################

# call data set
train, m_train = read_csv() 

#encode cat variable
train_df_enc = encode_cat(train)
m_train_df_enc = m_encode_cat(m_train)

#encode num variable & drop Null rows
train_df = encode_num(train_df_enc).dropna(how='any')

#merging 1 : macro with price_doc,timestamp
price_time_df = train[['timestamp', 'price_doc']]
m_train_df_enc_merge = pd.merge(m_train_df_enc, price_time_df, how='inner', on='timestamp')

#variable dividing train (dep, cat, num & dum)
dep_df = train_df[['id', 'timestamp', 'price_doc']]
cat_df = train_df[['material', 'sub_area', 'ecology', 'state', 'ID_big_road1', 'ID_big_road2', 'ID_bus_terminal', \
                   'ID_metro', 'ID_railroad_station_avto', 'ID_railroad_station_walk', 'ID_railroad_terminal']]
num_dum_df = train_df.drop([*dep_df.columns, *cat_df.columns], axis=1)
y = dep_df['price_doc']   
y_log = np.log(y)

#variable dividing macro
m_dep_df = m_train_df_enc_merge[['price_doc', 'timestamp']]
# m_cat_df = m_train_df_enc_merge[['child_on_acc_pre_school', 'modern_education_share', 'old_education_build_share']]
m_num_dum_df = m_train_df_enc_merge.drop([ *m_dep_df.columns], axis=1) # *m_cat_df.columns,
m_y = m_dep_df.price_doc

#num_dum_df, dep outliers cleasing & cat_var rows cleansing following num_dum_df  :  train
num_dum_df_do1, y_do1 = dropping_outliers1(num_dum_df, y)
cat_df_do1 = cat_df.loc[num_dum_df_do1.index, :]
dep_df_do1 = dep_df.loc[num_dum_df_do1.index, :]

#num_dum_df, dep outliers cleasing & cat_var rows cleansing following num_dum_df  :  macro
m_num_dum_df_do1, m_y_do1 = dropping_outliers1(m_num_dum_df, m_y)
# m_cat_df_do1 = m_cat_df.loc[m_num_dum_df_do1.index, :]
m_dep_df_do1 = m_dep_df.loc[m_num_dum_df_do1.index, :]

#merging 2 : train with macro
temp_for_mgr_num_dum_df_do1 = pd.concat([num_dum_df_do1, dep_df_do1], axis=1)
temp_for_mgr_m_num_dum_df_do1 = pd.concat([m_num_dum_df_do1, cat_df_do1, m_dep_df_do1], axis=1)
temp_for_mgr_m_num_dum_df_do1_grouped = temp_for_mgr_m_num_dum_df_do1.groupby('timestamp').aggregate(np.mean).reset_index().drop(['price_doc'],  axis=1)
train_mrg_macro_df = pd.merge(temp_for_mgr_num_dum_df_do1, temp_for_mgr_m_num_dum_df_do1_grouped, how='left', on='timestamp')

#variable dividing total
t_dep_df = train_mrg_macro_df[['id', 'timestamp', 'price_doc']]
t_cat_df = train_mrg_macro_df[['material', 'sub_area', 'ecology', 'state', 'ID_big_road1', 'ID_big_road2', 'ID_bus_terminal', \
                   'ID_metro', 'ID_railroad_station_avto', 'ID_railroad_station_walk', 'ID_railroad_terminal']]
t_num_dum_df = train_mrg_macro_df.drop([*t_cat_df.columns, *t_dep_df.columns], axis=1)
t_y = t_dep_df[['price_doc']]

print(train_mrg_macro_df.shape)

Function dropping_outliers :250 rows have beed deleted
Function dropping_outliers :279 rows have beed deleted
(4700, 391)


#    
#   Contents
##   1. Train part
##   2. Macro part

#      
#      
# Train Part
#    
#     

# Automation of 
 auto_eli_and_result_print를 사용하실 경우 print문을 막고 진행하시길 바랍니다.
 너무 무거워서 50이상일 경우 오래걸림.

In [24]:
auto_result_dict = {}
for i in range(2):
    model = 0; ser_1 = np.random.random_sample()/100; ser_2 = np.random.random_sample()*6/10 + 0.4; 
    ser_3 = np.random.random_sample()/100; lim_ = np.random.random_sample()*5/100
    
    x, y, result, model = auto_eli_and_result_print(num_dum_df_do1, y_do1, model, ser_1, ser_2, ser_3, lim_)
    temp_list = [ser_1, ser_2, ser_3, len(result.params.index), result.rsquared, result.rsquared_adj, result.condition_number]
    auto_result_dict[i] = temp_list

/home/dockeruser/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/home/dockeruser/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/dockeruser/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/dockeruser/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


KeyboardInterrupt: 

In [26]:
x1, y1, result1, model1 = auto_eli_and_result_print(num_dum_df_do1, y_do1, model=0, ser_1=0.000001, ser_2=0.5, ser_3=0.000001)

/home/dockeruser/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/home/dockeruser/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/dockeruser/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/dockeruser/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [11]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              price_doc   R-squared:                       0.639
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     331.4
Date:                Mon, 23 Oct 2017   Prob (F-statistic):               0.00
Time:                        15:51:21   Log-Likelihood:                -76880.
No. Observations:                4707   AIC:                         1.538e+05
Df Residuals:                    4681   BIC:                         1.540e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

In [27]:
print(result1.summary())
result1_params = list(result1.params.index)
result1_params

                            OLS Regression Results                            
Dep. Variable:              price_doc   R-squared:                       0.637
Model:                            OLS   Adj. R-squared:                  0.635
Method:                 Least Squares   F-statistic:                     547.0
Date:                Mon, 23 Oct 2017   Prob (F-statistic):               0.00
Time:                        16:02:04   Log-Likelihood:                -76774.
No. Observations:                4700   AIC:                         1.536e+05
Df Residuals:                    4684   BIC:                         1.537e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

['const',
 'full_sq',
 'kitch_sq',
 'product_type',
 'kremlin_km',
 'build_year',
 'max_floor',
 'floor',
 'leisure_count_500',
 'build_count_1971-1995',
 'build_count_1946-1970',
 'cafe_sum_1000_max_price_avg',
 'nuclear_reactor_km',
 'ttk_km',
 'mosque_count_1500',
 'kindergarten_km']

# Variable drop, 1 by 1

In [81]:
tr1_x = num_dum_df_do1[['full_sq', 'kitch_sq', 'product_type', 'kremlin_km',
       'build_year', 'max_floor', 'floor', 'leisure_count_500',
       'build_count_1971_1995', 'cafe_sum_1000_max_price_avg', 'nuclear_reactor_km', 'ttk_km',
       'mosque_count_1500', 'kindergarten_km']]
tr1_x1 = sm.add_constant(tr1_x)
result = sm.OLS(y_do1, tr1_x1).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              price_doc   R-squared:                       0.638
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     589.8
Date:                Mon, 23 Oct 2017   Prob (F-statistic):               0.00
Time:                        17:06:39   Log-Likelihood:                -76766.
No. Observations:                4700   AIC:                         1.536e+05
Df Residuals:                    4685   BIC:                         1.537e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

# R-Style function
 r_style_from_formula_enc_included 

In [ ]:
r_result1, r_model1 = r_style_from_formula_enc_included(tr1_x, cat_df_do1, y_do1, const=1, scal=0)


In [2]:
r_result1, r_model1 = r_style_from_formula_enc_included(tr1_x, cat_df_do1[['ecology', 'material']], y_do1, const=1, scal=1)
print(r_result1.summary())



r_temp_df = pd.concat([tr1_x1, cat_df_do1[['ecology', 'material']]], axis=1)
r_temp_df_x1 = sm.add_constant(r_temp_df)
y_new = r_result1.predict(r_temp_df_x1)
rmsle(y_do1, y_new)
np.sqrt(metrics.mean_squared_error(y_do1, y_new))    


# temp_df = pd.concat([num_dum_df[['full_sq', 'kitch_sq', 'floor', 'product_type', 'cafe_count_1500_price_4000', 'kremlin_km', 'leisure_count_500', 'build_count_slag']], cat_df], axis=1)
# temp_df1 = sm.add_constant(temp_df)
# y_new = result.predict(temp_df1)
# rmsle(y, y_new)

NameError: name 'r_style_from_formula_enc_included' is not defined

In [1]:
r_result1.params

NameError: name 'r_result1' is not defined

#    
#     
#  Macro Part
#     
#     

In [12]:
m_x1, m_y1, m_result1, m_model1 = auto_eli_and_result_print(m_num_dum_df_do1, m_y_do1, model=0, ser_1=0.001, ser_2=0.6, ser_3=0.05)

KeyboardInterrupt: 

In [46]:
print(m_result1.summary())

                            OLS Regression Results                            
Dep. Variable:              price_doc   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     126.8
Date:                Mon, 23 Oct 2017   Prob (F-statistic):          3.33e-107
Time:                        19:14:25   Log-Likelihood:            -3.5097e+05
No. Observations:               21291   AIC:                         7.019e+05
Df Residuals:                   21286   BIC:                         7.020e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

#    
#     
#  Total Data Part
#     
#     

In [5]:
t_x1 = sm.add_constant(t_num_dum_df)
result = sm.OLS(t_y, t_x1).fit()

In [7]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              price_doc   R-squared:                       0.749
Model:                            OLS   Adj. R-squared:                  0.732
Method:                 Least Squares   F-statistic:                     45.77
Date:                Mon, 23 Oct 2017   Prob (F-statistic):               0.00
Time:                        21:46:30   Log-Likelihood:                -75910.
No. Observations:                4700   AIC:                         1.524e+05
Df Residuals:                    4412   BIC:                         1.543e+05
Df Model:                         287                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

In [14]:
t_x1, t_y1, t_result1, t_model1 = serial_eli_lim_pval(t_num_dum_df, t_y, ser_1=0.01, ser_2=0.45, ser_3=0.001)

ValueError: labels ['const'] not contained in axis

# For function check part

In [ ]:
result_x = []
x1 = sm.add_constant(num_dum_df_do1)
model = sm.OLS(y_do1, x1).fit()
sorted_pvalues = model.pvalues.sort_values(ascending=True)
sorted_pvalues.drop(['const'], axis=0)
sorted_pvalues_lst = list(sorted_pvalues.index)
        
x1 = x1.drop(['const'], axis=1)
x1_cor = x1.corr()
x1_cor_eli_lst = list(x1_cor[abs(x1_cor[sorted_pvalues_lst[0]]) > 0.5].index)
x1_cor_eli_lst.remove(sorted_pvalues_lst[0])
x1_pval_eli_lst = list(sorted_pvalues[x1_cor_eli_lst][sorted_pvalues[x1_cor_eli_lst] > 0.05].index)
x1 = x1.drop([*x1_pval_eli_lst, sorted_pvalues_lst[0]], axis=1)
        
result_x.append(sorted_pvalues_lst[0])
remain_var_num = len(x1.columns)
'const' in sorted_pvalues.index

In [ ]:
x, y, result, model = auto_eli_and_result_print(num_dum_df_do1, y_do1, model = 0, ser_1 = 0.00001, ser_2 = 0.7, ser_3 = 0.0005)

In [ ]:
m_num_dum_df_do1, m_y_do1 = dropping_outliers1(m_num_dum_df, m_y)